In [1]:
#original code source from class: https://colab.research.google.com/drive/1hqefqT4VkHhSogO7AuyGHN8bEopnxLns?usp=sharing
import random
import requests
from tkinter import *

url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

#creating a canvas widget to show the trivia question
canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.place(relx=.5, rely=.5,anchor=CENTER)

#adds text to the canvas
game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

#creating a label to show if the user answer is correct or not
feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

#creating a string variable that hold the user's answer
user = StringVar()
user.set("")
#set the score and count of questions to zero that can be counted later on
score = 0
count_question = 0

#function to generate a list of  questions
def generate_qlist(json):
    return [i["question"] for i in json]

#function to randomly select a question from the API list
def gen_question(list_):
    return random.choice(list_)

#function to generate a list of options for a each question
def generate_answers(json, question): 
    for i in json:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list = [cor] + i["incorrectAnswers"]
            random.shuffle(option_list)
            return option_list

#function to ask the trivia question
def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)
    gen_radiobuttons(answers)

#function to make radio buttons for each question option
def gen_radiobuttons(answers):
    y_pos = 100
    for i in answers:
        rad_btn = Radiobutton(canvas, text=i, variable=user, command=grade_score, value=i, font=('Helvetica 10 bold'))
        rad_btn.place(x=50, y=y_pos)
        #The radiobutton is at the x-coordinate of 50, and the y-coordinate is goes by 20 so that the next radiobutton will be placed below
        y_pos += 20

#function to evaluate the user's answer and update the score
#adds 1 to score variable when a user selects the correct answer, and will say "correct" or "Incorrect"
#There are 10 questions, at the end it will print how many user got right out of 10
def grade_score():
    global count_question, score
    count_question += 1
    answer = user.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                feedback["text"] = "Correct!"
                feedback.place(x=400, y=500)
                score += 1
            else:
                feedback["text"] = "incorrect :("
                feedback.place(x=400, y=500)
    #10 questions to be counted
    if count_question >= 10:
        show_score()
        #disable "next" button once the user has answer 10 questions
        btn_next.config(state=DISABLED)
    else:
        main()

#function to display the final score
def show_score():
    score_label = Label(root, text=f"You got {score} out of 10 questions right!", font=("arial", 20), pady=20)
    score_label.pack()

#function to ask another question
def main(): 
    global current_question
    q_list = generate_qlist(get_request)
    current_question = gen_question(q_list)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)

#button to click next
btn_next = Button(root, text="Next", command=grade_score)
btn_next.pack()

main()

root.mainloop()



